# 휴일당번 약국 크롤러

- https://www.pharm114.or.kr/main.asp

In [1]:
!pip install selenium

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [1]:
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.alert import Alert
from bs4 import BeautifulSoup

from collections import defaultdict
import time
import datetime
import os
import json
import re


In [220]:
gu_ls = [ 'nowon_gu', 'jongno_gu', 'seongbuk_gu', 'kangbuk_gu', 'dobong_gu', 'seocho_gu',
'dongdaemun_gu', 'jungnang_gu', 'jung_gu', 'seongdong_gu', 'kwangjin_gu',
'kangdong_gu', 'mapo_gu', 'seodaemun_gu', 'kangseo_gu', 'yongsan_gu', 'kangnam_gu',
'songpa_gu', 'yeongdungpo_gu', 'dongjak_gu', 'guro_gu', 'yangcheon_gu', 'gwanak_gu',
'gumcheon_gu']

In [226]:

driver = webdriver.Chrome('./chromedriver.exe')
da = Alert(driver)
driver.implicitly_wait(10)
driver.get('https://www.pharm114.or.kr/common_files/sub2_page2.asp?addr1=%BC%AD%BF%EF%C6%AF%BA%B0%BD%C3')
driver.find_element_by_xpath('//*[@class="' + 'dobong_gu' + '"]/a').click()
da.accept()
driver.implicitly_wait(10)
time.sleep(5)
driver.find_element_by_xpath('//a[2]').click()
driver.implicitly_wait(10)
bound_p1 = re.search(r'\b=\b',driver.current_url).start() + 1
bound_p2 = re.search(r'\b&\b',driver.current_url).start()

    
hour_df = pd.DataFrame([], columns = ['hours'])
df = pd.DataFrame([], columns = ['name', 'address', 'phone'])
sat_total_df = dict()

j_index = -1

for k in range(1, 200):
    driver.get(driver.current_url[:bound_p1] + str(k) + driver.current_url[bound_p2:])
    tmp_html = driver.page_source
    
    if BeautifulSoup(tmp_html, 'html.parser').find('td', {'class' : 'class_addr'}) is not None:
        html = driver.page_source
        
    if BeautifulSoup(tmp_html, 'html.parser').find('td', {'class' : 'class_addr'}) is None:
        break
    
    lis = BeautifulSoup(html, 'html.parser').find('div', {'id':'printZone'})
    a = lis.find('script')
    opr_hour_ls = []

    for i in range(1,25,1):
    #print(a)
        if a.find_next('td') is None:
            break
    
        cleaned_hour = re.sub('\t', '', a.find_next('td').find_next('td').text)
        cleaned_hour = re.sub('\n', '', cleaned_hour)
    
        opr_hour_ls.append(cleaned_hour)
        a = a.find_next('script')
    
    one_page_hour_df = pd.DataFrame(opr_hour_ls, columns = ['hours'])
    
    hour_df = pd.concat([hour_df, one_page_hour_df], axis = 0)
    
    soup = BeautifulSoup(html, 'html.parser').find_all('a')
    
    for i in range(15, 52, 1):
        j_index += 1
        
        try :
            one_link = str(soup[i])
        
            if re.search(r'company_name=', one_link) is None:
                continue
        
            bound1 = re.search(r'company_name=', one_link).end()
            bound2 = re.search(r'&amp;phone', one_link).start()
            one_pharm = pd.DataFrame([], columns = ['name', 'address', 'phone'])
            one_pharm.loc[j_index, 'name'] = one_link[bound1 : bound2]
        
        
            bound3 = re.search(r'addr=', one_link).end()
            bound4 = re.search(r'&amp;', one_link).start()
            one_pharm.loc[j_index, 'address'] = one_link[bound3 : bound4]


            bound5 = re.search(r'phone=', one_link).end()
            bound6 = re.search(r"<img border", one_link).start() - 4
            one_pharm.loc[j_index, 'phone'] = one_link[bound5 : bound6]
            
            
            df = pd.concat([df, one_pharm])
            df['imageSrc'] = "/images/phar.png"
                
        except IndexError :
            pass
        
sat_total_df['dobong_gu'] = pd.concat([df.reset_index(drop=True), hour_df.reset_index(drop=True)], axis=1)
sat_total_df['dobong_gu'].to_csv('dobong_gu'+'.csv', index = False )

c:\python\python37\lib\site-packages\ipykernel_launcher.py:78: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [228]:
sat_total_df['dobong_gu']

,address,imageSrc,name,phone,hours
0,서울특별시 도봉구 마들로 645 1층,/images/phar.png,오약국,02-955-0788,09:30 ~ 15:00
1,서울특별시 도봉구 도봉로 468,/images/phar.png,백운약국,02-906-0280,08:30 ~ 19:00
2,서울특별시 도봉구 우이천로4길 32,/images/phar.png,녹십자약국,02-992-9266,09:00 ~ 15:00
3,서울특별시 도봉구 해등로 190상가1층,/images/phar.png,건강제일녹십자약국,02-906-1024,09:00 ~ 18:00
4,서울특별시 도봉구 시루봉로15길 36,/images/phar.png,태양당약국,02-3493-0664,09:00 ~ 21:00
5,서울특별시 도봉구 도봉로 575,/images/phar.png,일충프라자약국,02-902-9696,09:00 ~ 15:00
6,서울특별시 도봉구 노해로65길 14-3,/images/phar.png,종로프라자약국,02-907-4266,08:30 ~ 22:00
7,서울특별시 도봉구 도봉로 476206호,/images/phar.png,삼성늘푸른약국,02-993-5757,08:00 ~ 16:00
8,서울특별시 도봉구 노해로63길 84-3,/images/phar.png,창동종로약국,02-992-2376,08:00 ~ 21:30
9,서울특별시 도봉구 도당로13길 16,/images/phar.png,햇빛약국,02-6081-5255,09:00 ~ 16:00
